In [2]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.utils.data as tud
from tensorboardX import SummaryWriter
from torchviz import make_dot
import tensorwatch as tw
import time

# 转换方式

In [25]:

transform = transforms.Compose( [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_dataloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

validset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
valid_dataloader = torch.utils.data.DataLoader(validset, batch_size=32, shuffle=False, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_dataloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# 神经网络 

In [2]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [3]:
class MyNet(nn.Module):
    def __init__(self,num_features,hidden_size,output_size):
        super(MyNet, self).__init__()
        self.conv1 = nn.Conv2d(num_features, 20, 5, 1)
        self.bn1 = nn.BatchNorm2d(20)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.bn2 = nn.BatchNorm2d(50)
        self.fc1 = nn.Linear(5*5*50, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x):
        # x:1 * 28 * 28
        
        x = self.bn1(self.conv1(x))
        x = F.ReLU(x) # 20 * 28 * 28
        x = F.max_pool2d(x, 2, 2) # 20 * 14 * 14
        
        x = self.dropout(x)
        x = self.bn2(self.conv2(x))
        x = F.ReLU(x) # 20 * 28 * 28
        x = F.max_pool2d(x, 2, 2) # 50 * 5 * 5
        
        x = x.view(-1, 5*5*50) #reshape
        x1 = F.ReLU(self.fc1(x))
        
        x1 = self.dropout(x1)
        x1 = self.fc2(x1)
        
        return F.log_softmax(x1, dim = 1) # log probability

In [7]:
model = MyNet(num_features = 3,hidden_size = 500,output_size = 10)
count_parameters(model)

657220

In [17]:
dummy_input = torch.rand(20, 3, 32, 32)
model = MyNet(num_features = 3,hidden_size = 100,output_size = 10)
with SummaryWriter(comment = 'MyNet') as w:
    w.add_graph(model,(dummy_input,))

In [3]:
class Res_Net(nn.Module):

    def __init__(self):
        super(Res_Net,self).__init__()
        
        self.layer0 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        )
        
       
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
            nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        
        self.res_connect2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=1, stride=2, bias=False),
            nn.BatchNorm2d(128)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
            nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        
        self.res_connect3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=1, stride=2, bias=False),
            nn.BatchNorm2d(256)
        )
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False),
            nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
            nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        
        self.res_connect4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=1, stride=2, bias=False),
            nn.BatchNorm2d(512)
        )
        
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.fc = nn.Linear(in_features=512, out_features=10, bias=True)

        

    def forward(self,x):
        x = self.layer0(x)
        
        x = self.layer1(x) + x
        x = self.layer2(x) + self.res_connect2(x)
        x = self.layer3(x) + self.res_connect3(x)
        x = self.layer4(x) + self.res_connect4(x)
        
        x = self.avgpool(x)
        x = x.view(-1,512)
        x = self.fc(x)

        return F.softmax(x,dim=1)

In [6]:
model = Res_Net()
count_parameters(model)

4737098

In [20]:
dummy_input = torch.rand(20, 3, 32, 32)
model = Res_Net()
with SummaryWriter(comment = 'Res_Net') as w:
    w.add_graph(model,(dummy_input,))

In [13]:
class VGG_Net(nn.Module):

    def __init__(self):
        super(VGG_Net,self).__init__()
        
        self.layer0 = nn.Sequential(
            nn.Conv2d(3,64,3,padding=1),
            nn.Conv2d(64,64,3,padding=1),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(64,128,3,padding=1),
            nn.Conv2d(128, 128, 3,padding=1),
            nn.MaxPool2d(2, 2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU()
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(128,128, 3,padding=1),
            nn.Conv2d(128, 128, 3,padding=1),
            nn.Conv2d(128, 128, 1,padding=1),
            nn.MaxPool2d(2, 2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU()
        )
        

        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 256, 3,padding=1),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.Conv2d(256, 256, 1, padding=1),
            nn.MaxPool2d(2, 2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU()
        )
        
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(256, 512, 3, padding=1),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.Conv2d(512, 512, 1, padding=1),
            nn.MaxPool2d(2, 2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        
        
        self.layer5 = nn.Sequential(
            nn.Linear(512*4*4,1024),
            nn.ReLU(),
            nn.Dropout2d(),
            nn.Linear(1024,1024),
            nn.ReLU(),
            nn.Dropout2d(),
            nn.Linear(1024,10)
        )
        
        
    def forward(self,x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = x.view(-1,512*4*4)
        
        x = self.layer5(x)
        
        return F.softmax(x)

In [14]:
model = VGG_Net()
count_parameters(model)

14777162

In [22]:
dummy_input = torch.rand(20, 3, 32, 32)
model = VGG_Net()
with SummaryWriter(comment = 'Res_Net') as w:
    w.add_graph(model,(dummy_input,))